<a href="https://colab.research.google.com/github/johnsonjzhou/comp90089-project/blob/main/sql/initial_cohort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COMP90089 Final Project**

## **Selecting Initial Patient Cohort from MIMICIV**

In [1]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive'

Mounted at /content/drive


In [ ]:
!pip install -U pandasql

In [3]:
#Set up the environement

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functools as ft
from pandasql import sqldf
%matplotlib inline


#Project_ID
project_id = "mimic-iv-projects" 

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
    return pd.io.gbq.read_gbq(
        query,
        project_id=project_id,
        dialect='standard')


### **SQL queries**

In [4]:
#Select patients from ICU_stays based on: n_stay >= 1 and average los >= 3.3 per 	hadm_id:

c1= f"""
SELECT
    icustays.subject_id As subject_id,
    icustays.hadm_id As hadm_id,
    count(icustays.stay_id) As n_stays,
    avg(icustays.los) As avg_los
FROM
  `physionet-data.mimiciv_icu.icustays` AS icustays
GROUP BY
    subject_id,	hadm_id
HAVING 
   n_stays >= 1 AND avg_los >= 3.3
ORDER BY
    subject_id,	hadm_id
"""

In [169]:
cohort_c1 = f"""
    
    SELECT  icustays.subject_id As subject_id,
            icustays.hadm_id As hadm_id,
            icustays.stay_id As stay_id,
           c1.n_stays,
           c1.avg_los,
    FROM  ({c1}) As c1
    INNER JOIN `physionet-data.mimiciv_icu.icustays` AS icustays
    ON icustays.hadm_id = c1.hadm_id
"""

In [170]:
#Select patients from cohort_c1 based on the age range (18 - 90) and survival status (dod is null) criteria from Table: hosp.patients :
cohort_q1= f"""
      SELECT cohort_c1.*,
            patients.gender As gender,
            patients.anchor_age As anchor_age
      FROM ({cohort_c1}) As cohort_c1
      INNER JOIN `physionet-data.mimiciv_hosp.patients` As patients
        ON cohort_c1.subject_id = patients.subject_id AND (patients.anchor_age BETWEEN 18 AND 90) AND  (patients.dod IS NULL)
        ORDER BY
          patients.subject_id
        """

In [171]:
#Demographic_1
# Presence of infection
# Table: mimiciv_derived.suspicion_of_infection
# Presence of infection (suspected_infection = 1)


demographic_1 = f"""
    SELECT suspicion_of_infection.subject_id As subject_id,
          suspicion_of_infection.suspected_infection As suspected_infection,
    FROM `physionet-data.mimiciv_derived.suspicion_of_infection` As suspicion_of_infection
"""

In [172]:
#Demographic_2
# Table: mimiciv_hosp.admissions
# Type of admission (admission_type)

#Note: Some patients have more than one admission type

demographic_2 = f"""
    SELECT  DISTINCT admission.subject_id As subject_id,
            admission.hadm_id As hadm_id,
            admission.admission_type As admission_type
    FROM `physionet-data.mimiciv_hosp.admissions` As admission
"""

In [173]:
##Demographic_3
# Table: mimiciv_derived.first_day_weight
# weight
# Table: mimiciv_derived.first_day_height
# height
demographic_3 = f"""
    SELECT DISTINCT first_day_weight.subject_id As subject_id,
            first_day_weight.stay_id As stay_id,
            first_day_weight.weight As weight,
            first_day_height.height As height,
    FROM `physionet-data.mimiciv_derived.first_day_weight` as first_day_weight
    LEFT JOIN `physionet-data.mimiciv_derived.first_day_height` As first_day_height
    ON first_day_weight.stay_id = first_day_height.stay_id
   """

In [174]:
#Cohort_Demographic
demographic = f"""
    SELECT DISTINCT demographic_2.*, 
                    demographic_1.suspected_infection,
                    demographic_3.weight,
                    demographic_3.stay_id,
                    demographic_3.height
   FROM ({demographic_2}) As  demographic_2
   LEFT JOIN ({demographic_1}) As  demographic_1
     ON demographic_2.subject_id = demographic_1.subject_id
   LEFT JOIN ({demographic_3}) As  demographic_3
     ON demographic_1.subject_id = demographic_3.subject_id
    """

In [176]:
#Cohort_Demographic

cohort_q2 = run_query(f"""
        SELECT DISTINCT cohort_q1.*,
                        demographic.admission_type,
                        demographic.suspected_infection,
                        demographic.weight,
                        demographic.height
        FROM ({cohort_q1}) As cohort_q1
        LEFT JOIN ({demographic}) As demographic
         ON cohort_q1.hadm_id = demographic.hadm_id
        """)

In [177]:
cohort_q2.to_csv('/content/drive/MyDrive/cohort_criteria_data.csv')

In [178]:
# Vital signs
#Table: mimiciv_derived.first_day_vitalsign`

#heart rate(heart_rate_mean)
#respiratory_rate (resp_rate_mean)
#body_temp (temperature_mean)

# Invasive mean arterial pressure (mbp_mean) 
# Invasive systolic blood pressure (sbp_mean) (min and max)
# Invasive diastolic blood pressure (dbp_mean) (min and max)
# Oxygen saturation (SPO2_mean)

#(Non-invasive values) not included in the first_day_vitalsign
# Non-invasive mean arterial pressure
# Non-invasive Systolic blood pressure
# Non-invasive Diastolic blood pressure

cohort_vitalsigns = run_query(f"""
SELECT DISTINCT 
       first_day_vitalsigns.subject_id As subject_id,
       first_day_vitalsigns.stay_id As stay_id,
       first_day_vitalsigns.heart_rate_mean As heart_rate_mean,
       first_day_vitalsigns.resp_rate_mean As resp_rate_mean,
       first_day_vitalsigns.temperature_mean As temperature_mean,
       first_day_vitalsigns.mbp_mean As mbp_mean,
       first_day_vitalsigns.sbp_mean As sbp_mean,
       first_day_vitalsigns.dbp_mean As dbp_mean,
       first_day_vitalsigns.spo2_mean As spo2_mean
 FROM `physionet-data.mimiciv_derived.first_day_vitalsign`  As first_day_vitalsigns 
""")

In [179]:
cohort_vitalsigns.to_csv('/content/drive/MyDrive/cohort_vitalsigns.csv')

In [ ]:

# Laboratory tests


#Table: mimiciv_derived.first_day_lab

# Albumin(albumin_min, albumin_max)
# Blood urea nitrogen (bun_min, bun_max)
# Calcium (calcium_min, calcium_max)
# creatinine (creatinine_min, creatinine_max)
# Glucose (glucose_min, glucose_max)
# Bicarbonate (bicarbonate_min, bicarbonate_max)
# Potassium (potassium_min, potassium_max)
# Sodium (sodium_min, sodium_max)
# Platelets (platelets_min, platelets_max)
# Bilirubin (bilirubin_total_min, bilirubin_total_max)
# White blood cell count (wbc_min, wbc_max)

#Table: mimiciv_derived.first_day_bg

# Lactate (lactate_min, lactate_max)
# pH (ph_min, ph_max)
# Partial pressure of carbon dioxide (pco2_min, pco2_max)
# Partial pressure of Oxygen (po2_min, po2_max)
# PaO2/FiO2 ratio (pao2fio2ratio_min, pao2fio2ratio_max)

#Not Found! 

# Lactate Dehydrogenase,
# Magnesium (Mg), 
#Leukocytes
# Urea

cohort_lab_tests = run_query(f"""
SELECT DISTINCT 
       lab_tests.subject_id As subject_id,
       lab_tests.stay_id As stay_id,
       lab_tests.albumin_min As albumin_min,
       lab_tests.albumin_max As albumin_max,
       lab_tests.bun_min As bun_min,
       lab_tests.bun_max As bun_max,
       lab_tests.calcium_min As calcium_min,
       lab_tests.calcium_max As calcium_max,
       lab_tests.creatinine_min As creatinine_min,
       lab_tests.creatinine_max As creatinine_max,
       lab_tests.glucose_min As glucose_min,
       lab_tests.glucose_max As glucose_max,
       lab_tests.bicarbonate_min As bicarbonate_min,
       lab_tests.bicarbonate_max As bicarbonate_max,
       lab_tests.potassium_min As potassium_min,
       lab_tests.potassium_max As potassium_max,
       lab_tests.sodium_min As sodium_min,
       lab_tests.sodium_max As sodium_max,
       lab_tests.platelets_min As platelets_min,
       lab_tests.platelets_max As platelets_max,
       lab_tests.bilirubin_total_min As bilirubin_total_min,
       lab_tests.bilirubin_total_max As bilirubin_total_max,
       lab_tests.wbc_min As wbc_min,
       lab_tests.wbc_max As wbc_max,
       lab_bg_tests.lactate_min As lactate_min,
       lab_bg_tests.lactate_max As lactate_max,
       lab_bg_tests.ph_min As ph_min,
       lab_bg_tests.ph_max As ph_max,
       lab_bg_tests.pco2_min As pco2_min,
       lab_bg_tests.pco2_max As pco2_max,
       lab_bg_tests.po2_min As po2_min,
       lab_bg_tests.po2_max As po2_max,
       lab_bg_tests.pao2fio2ratio_min As pao2fio2ratio_min,
       lab_bg_tests.pao2fio2ratio_max As pao2fio2ratio_max
      
 FROM `physionet-data.mimiciv_derived.first_day_lab`  As lab_tests
 INNER JOIN `physionet-data.mimiciv_derived.first_day_bg` As lab_bg_tests
 ON lab_tests.stay_id = lab_bg_tests.stay_id
""")


In [180]:
cohort_lab_tests.to_csv('/content/drive/MyDrive/cohort_lab_tests.csv')

In [181]:
# Comorbidities

# Table: mimiciv_derived.charlson
# Charlson index (charlson_comorbidity_index)


cohort_comorbidities_1 = f"""
SELECT DISTINCT 
       charlson_comorbidity.subject_id As subject_id,
       charlson_comorbidity.charlson_comorbidity_index As charlson
FROM `physionet-data.mimiciv_derived.charlson` As charlson_comorbidity
"""

In [182]:
# Table: mimiciv_derived.gcs
# Glasgow coma scale (gcs)

# Table: mimiciv_derived.first_day_sofa
# Sequential Organ Failure Assessment score (SOFA)

cohort_comorbidities_2 = f"""
SELECT DISTINCT 
       gcs_first_day.subject_id As subject_id,
       gcs_first_day.stay_id As stay_id,
       gcs_first_day.gcs_min As gcs,
       first_day_sofa.SOFA As sofa

FROM `physionet-data.mimiciv_derived.first_day_gcs` As gcs_first_day
INNER JOIN `physionet-data.mimiciv_derived.first_day_sofa` As first_day_sofa
ON gcs_first_day.stay_id = first_day_sofa.stay_id
"""

In [183]:
cohort_comorbidities = run_query(f"""
SELECT DISTINCT 
       cohort_comorbidities_2.*,
       cohort_comorbidities_1.charlson
FROM ({cohort_comorbidities_2}) as cohort_comorbidities_2
INNER JOIN ({cohort_comorbidities_1}) As cohort_comorbidities_1
ON cohort_comorbidities_2.subject_id = cohort_comorbidities_1.subject_id 
"""   
)

In [184]:
cohort_comorbidities.to_csv('/content/drive/MyDrive/cohort_comorbidities.csv')

In [185]:
# Device use

# Table: mimiciv_derived.ventilation
# Ventilation status (ventilation_status)

cohort_device_use = run_query(f"""
SELECT DISTINCT 
       cohort_q1.*,
       ventilation.ventilation_status As ventilation_status
FROM ({cohort_q1}) As cohort_q1
LEFT JOIN `physionet-data.mimiciv_derived.ventilation` As ventilation
ON ventilation.stay_id = cohort_q1.stay_id
""")

In [186]:
cohort_device_use.to_csv('/content/drive/MyDrive/cohort_device_use.csv')

In [187]:
# Input/output: 


# Table: mimiciv_derived.first_day_urine_output 
# Urine output (urineoutput)


cohort_urine_output = run_query(f"""
SELECT urine_output.subject_id As subject_id,
       urine_output.stay_id As stay_id,
       urine_output.urineoutput As urineoutput,           
FROM `physionet-data.mimiciv_derived.first_day_urine_output` As urine_output
""")

In [188]:
cohort_urine_output.to_csv('/content/drive/MyDrive/cohort_urine_output.csv')

In [189]:
# Input/output: 

# Table: mimiciv_derived.vasopressin 
# Use of vasopressors (vaso_rate)

cohort_vasopressin = run_query(f"""
SELECT DISTINCT
       vasopressin.stay_id As stay_id,
       vasopressin.vaso_rate As vaso_rate
FROM `physionet-data.mimiciv_derived.vasopressin` As vasopressin    
""")


In [190]:
cohort_vasopressin.to_csv('/content/drive/MyDrive/cohort_vasopressin.csv')

### **Join all result tables**

In [191]:
#Join tables: 

cohort_criteria = pd.read_csv('/content/drive/MyDrive/cohort_criteria_data.csv')
cohort_vitalsigns = pd.read_csv('/content/drive/MyDrive/cohort_vitalsigns.csv')
cohort_lab_tests = pd.read_csv('/content/drive/MyDrive/cohort_lab_tests.csv')
cohort_comorbidities = pd.read_csv('/content/drive/MyDrive/cohort_comorbidities.csv')
cohort_device_use = pd.read_csv('/content/drive/MyDrive/cohort_device_use.csv')
cohort_urine = pd.read_csv('/content/drive/MyDrive/cohort_urine_output.csv')
cohort_vasopressin = pd.read_csv('/content/drive/MyDrive/cohort_vasopressin.csv')


In [ ]:
cohort_criteria

In [192]:
#1. Join vital signs
query1 = f"""
SELECT DISTINCT cohort_criteria.*,
       cohort_vitalsigns.* 
FROM cohort_criteria  As cohort_criteria
LEFT JOIN cohort_vitalsigns AS cohort_vitalsigns
ON cohort_criteria.stay_id = cohort_vitalsigns.stay_id
""" 

In [193]:
#Run query
vitalsigns_joined  = sqldf(query1, globals())

In [194]:
#Remove duplicate Ids columns 
vitalsigns_joined = vitalsigns_joined.loc[:,~vitalsigns_joined.columns.duplicated()].copy()

In [195]:
#2. Join lab tests 
query2 = f"""
SELECT DISTINCT vitalsigns_joined.*,
       cohort_lab_tests.* 
FROM vitalsigns_joined  As vitalsigns_joined
LEFT JOIN cohort_lab_tests AS cohort_lab_tests
ON vitalsigns_joined.stay_id = cohort_lab_tests.stay_id
""" 

In [196]:
#Run query
lab_tests_joined  = sqldf(query2, globals())

In [197]:
#Remove duplicate Ids columns 
lab_tests_joined = lab_tests_joined.loc[:,~lab_tests_joined.columns.duplicated()].copy()

In [198]:
#3. Join comorbidities scores 

query3 = f"""
SELECT DISTINCT lab_tests_joined.*,
      cohort_comorbidities.* 
FROM lab_tests_joined  As lab_tests_joined
LEFT JOIN cohort_comorbidities AS cohort_comorbidities
ON lab_tests_joined.stay_id = cohort_comorbidities.stay_id
""" 

In [199]:
#Run query
comorbidities_joined  = sqldf(query3, globals())

In [200]:
#Remove duplicate Ids columns 
comorbidities_joined = comorbidities_joined.loc[:,~comorbidities_joined.columns.duplicated()].copy()

In [201]:
#4. Join used devices

query4 = f"""
SELECT DISTINCT comorbidities_joined.*,
      cohort_device_use.* 
FROM comorbidities_joined  As comorbidities_joined
LEFT JOIN cohort_device_use AS cohort_device_use
ON comorbidities_joined.stay_id = cohort_device_use.stay_id
""" 

In [202]:
#Run query
device_use_joined  = sqldf(query4, globals())

In [203]:
#Remove duplicate Ids columns 
device_use_joined = device_use_joined.loc[:,~device_use_joined.columns.duplicated()].copy()

In [204]:
#5. Join urine output

query5 = f"""
SELECT DISTINCT device_use_joined.*,
      cohort_urine.* 
FROM device_use_joined  As device_use_joined
LEFT JOIN cohort_urine AS cohort_urine
ON device_use_joined.stay_id = cohort_urine.stay_id
""" 

In [205]:
#Run query
urine_output_joined  = sqldf(query5, globals())

In [206]:
#Remove duplicate Ids columns 
urine_output_joined = urine_output_joined.loc[:,~urine_output_joined.columns.duplicated()].copy()

In [207]:
#6. Join vasopressin rate

query6 = f"""
SELECT DISTINCT urine_output_joined.*,
      cohort_vasopressin.* 
FROM urine_output_joined  As urine_output_joined
LEFT JOIN cohort_vasopressin AS cohort_vasopressin
ON urine_output_joined.stay_id = cohort_vasopressin.stay_id
""" 

In [208]:
#Run query
vasopressin_rate_joined  = sqldf(query6, globals())

In [209]:
#Remove duplicate Ids columns 
vasopressin_rate_joined = vasopressin_rate_joined.loc[:,~vasopressin_rate_joined.columns.duplicated()].copy()

In [210]:
vasopressin_rate_joined 

,Unnamed: 0,subject_id,hadm_id,stay_id,n_stays,avg_los,gender,anchor_age,admission_type,suspected_infection,...,po2_min,po2_max,pao2fio2ratio_min,pao2fio2ratio_max,gcs,sofa,charlson,ventilation_status,urineoutput,vaso_rate
0,0,10018328,23786647,31269608,1,7.702512,F,83,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,12.0,3,5,SupplementalOxygen,2130.0,NaN
1,0,10018328,23786647,31269608,1,7.702512,F,83,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,12.0,3,6,SupplementalOxygen,2130.0,NaN
2,1,10103795,20269407,30619907,1,5.601065,M,58,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,14.0,1,3,None,2175.0,NaN
3,1,10103795,20269407,30619907,1,5.601065,M,58,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,14.0,1,5,None,2175.0,NaN
4,2,10103795,20269407,30619907,1,5.601065,M,58,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,14.0,1,3,None,2175.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321312,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,6,InvasiveVent,1425.0,NaN
321313,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,6,SupplementalOxygen,1425.0,NaN
321314,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,7,InvasiveVent,1425.0,NaN
321315,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,7,SupplementalOxygen,1425.0,NaN


### **Final result table**

In [211]:
intial_cohort = vasopressin_rate_joined.copy()

In [212]:
intial_cohort.to_csv('/content/drive/MyDrive/initial_cohort_final.csv')

In [215]:
intial_cohort

,Unnamed: 0,subject_id,hadm_id,stay_id,n_stays,avg_los,gender,anchor_age,admission_type,suspected_infection,...,po2_min,po2_max,pao2fio2ratio_min,pao2fio2ratio_max,gcs,sofa,charlson,ventilation_status,urineoutput,vaso_rate
0,0,10018328,23786647,31269608,1,7.702512,F,83,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,12.0,3,5,SupplementalOxygen,2130.0,NaN
1,0,10018328,23786647,31269608,1,7.702512,F,83,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,12.0,3,6,SupplementalOxygen,2130.0,NaN
2,1,10103795,20269407,30619907,1,5.601065,M,58,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,14.0,1,3,None,2175.0,NaN
3,1,10103795,20269407,30619907,1,5.601065,M,58,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,14.0,1,5,None,2175.0,NaN
4,2,10103795,20269407,30619907,1,5.601065,M,58,SURGICAL SAME DAY ADMISSION,1.0,...,NaN,NaN,NaN,NaN,14.0,1,3,None,2175.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321312,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,6,InvasiveVent,1425.0,NaN
321313,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,6,SupplementalOxygen,1425.0,NaN
321314,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,7,InvasiveVent,1425.0,NaN
321315,39638,19985545,21516111,34458489,2,3.710492,M,56,EW EMER.,1.0,...,122.0,145.0,290.0,290.0,14.0,8,7,SupplementalOxygen,1425.0,NaN


In [216]:
intial_cohort.describe()

,Unnamed: 0,subject_id,hadm_id,stay_id,n_stays,avg_los,anchor_age,suspected_infection,weight,height,...,pco2_max,po2_min,po2_max,pao2fio2ratio_min,pao2fio2ratio_max,gcs,sofa,charlson,urineoutput,vaso_rate
count,321317.000000,3.213170e+05,3.213170e+05,3.213170e+05,321317.000000,321317.000000,321317.000000,319745.000000,308542.000000,165890.000000,...,279961.000000,279961.000000,279961.000000,235858.000000,235858.000000,320704.000000,321317.000000,321317.000000,310289.000000,160458.000000
mean,22220.355808,1.477356e+07,2.513722e+07,3.488207e+07,1.784272,11.950526,56.412533,0.544756,87.576721,171.057990,...,52.757184,70.963834,254.967817,148.830517,319.508463,10.482133,8.882238,5.454333,1730.545161,2.291594
std,10620.792255,2.963229e+06,2.846122e+06,2.890546e+06,1.277098,11.720023,14.486126,0.497994,26.143161,10.739245,...,17.722566,44.117805,151.352600,95.200412,170.524283,4.310523,4.630846,2.667516,1383.773149,3.769864
min,0.000000,1.000235e+07,2.000136e+07,3.000147e+07,1.000000,3.300185,18.000000,0.000000,1.000000,122.000000,...,8.000000,4.000000,14.000000,5.714286,29.000000,3.000000,0.000000,0.000000,0.000000,0.060000
25%,12676.000000,1.204384e+07,2.293473e+07,3.240427e+07,1.000000,4.734034,48.000000,0.000000,70.650000,163.000000,...,43.000000,39.000000,117.000000,75.000000,215.000000,7.000000,5.000000,4.000000,770.000000,1.201697
50%,24172.000000,1.460799e+07,2.520105e+07,3.481464e+07,1.000000,7.948472,58.000000,1.000000,84.850000,173.000000,...,49.000000,63.000000,231.000000,135.000000,292.000000,12.000000,9.000000,5.000000,1475.000000,2.400000
75%,31109.000000,1.727769e+07,2.746945e+07,3.734237e+07,2.000000,13.649931,67.000000,1.000000,100.000000,178.000000,...,57.000000,90.000000,379.000000,200.000000,385.000000,14.000000,12.000000,7.000000,2335.000000,2.406443
max,39639.000000,1.999944e+07,2.999962e+07,3.999981e+07,7.000000,101.726238,89.000000,1.000000,1251.000000,203.000000,...,199.000000,557.000000,795.000000,1302.500000,2242.857143,15.000000,21.000000,17.000000,30595.000000,295.350006
